In [3]:
import pandas as pd
import numpy as np

In [9]:
cd C:\Users\Diego\Documents\DTU\Spring2024\MBML\european_vote_analysis\

C:\Users\Diego\Documents\DTU\Spring2024\MBML\european_vote_analysis


In [35]:
data_national = pd.read_csv(r'data/raw/COMEPELDA_aggregate_v1.00.csv')
data_european = pd.read_csv(r'data/raw/1999-2019_CHES_dataset_means.csv')
country_codes = pd.read_csv(r'data/raw/countries.csv')

In [36]:
# Add country codes to national data
data_national = pd.merge(data_national, country_codes, left_on='cName', right_on='Country', how='left')

In [41]:
data_national

,IDAD,cName,ElYear,IDpty,pName,dName,dNameEng,cValid,dValid,pcAbs,...,pAddLisbon,pAlliance01,IDauxAlliance,pApparParty1,pApparParty2,IDauxElec,IDauxIndep,ID,Abbreviation,Country
0,1,Austria,1999,50,Freiheitliche Partei Österreichs,(nationwide),(nationwide),2801353,2801353,655519,...,0,0,.,.,.,16,.,13,AUS,Austria
1,2,Austria,1999,955,Liberales Forum,(nationwide),(nationwide),2801353,2801353,74467,...,0,0,.,.,.,16,.,13,AUS,Austria
2,3,Austria,1999,973,Sozialdemokratische Partei Österreichs,(nationwide),(nationwide),2801353,2801353,888338,...,0,0,.,.,.,16,.,13,AUS,Austria
3,4,Austria,1999,1013,Österreichische Volkspartei,(nationwide),(nationwide),2801353,2801353,859175,...,0,0,.,.,.,16,.,13,AUS,Austria
4,5,Austria,1999,1429,Die Grünen – Die Grüne Alternative,(nationwide),(nationwide),2801353,2801353,260273,...,0,0,.,.,.,16,.,13,AUS,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,1802,Sweden,2014,904,Socialdemokraterna,(nationwide),(nationwide),3716778,3716778,899074,...,0,0,.,.,.,843,.,16,SV,Sweden
1802,1803,Sweden,2014,1154,Miljöpartiet de Gröna,(nationwide),(nationwide),3716778,3716778,572591,...,0,0,.,.,.,843,.,16,SV,Sweden
1803,1804,Sweden,2014,1461,Centerpartiet,(nationwide),(nationwide),3716778,3716778,241101,...,0,0,.,.,.,843,.,16,SV,Sweden
1804,1805,Sweden,2014,1521,Feministiskt initiativ,(nationwide),(nationwide),3716778,3716778,204005,...,0,0,.,.,.,843,.,16,SV,Sweden


In [42]:
data_european

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,mip_one,mip_two,mip_three,chesversion
0,1,1,1,1999,9,115,NaN,FN,1.50,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.1
1,1,1,1,1999,9,109,21521.0,CVP,14.10,14.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.1
2,1,1,1,1999,9,107,21421.0,PVV/VLD,14.30,15.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.1
3,1,1,1,1999,9,106,21422.0,PRL,7.70,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.1
4,1,1,1,1999,9,110,21913.0,VU,5.60,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,40,0,1,2019,2,4005,55322.0,EDEK,6.18,5.4,...,0.0,3.0,3.5,4.5,6.0,8.5,NaN,NaN,NaN,2020.1
1192,40,0,1,2019,2,4006,55110.0,KOP,4.81,3.6,...,0.0,2.0,4.0,6.0,8.0,5.5,NaN,NaN,NaN,2020.1
1193,40,0,1,2019,2,4004,55422.0,DIKO,14.49,16.1,...,0.0,2.5,4.0,3.0,6.5,8.5,NaN,NaN,NaN,2020.1
1194,40,0,1,2019,2,4009,NaN,ELAM,3.71,3.6,...,0.0,8.5,6.5,6.5,6.0,9.5,NaN,NaN,NaN,2020.1
